# Change-Point Detection for Safety Regression

This notebook demonstrates the change-point model for detecting shifts in hazard rates.

## Objectives
1. Prepare time series data
2. Fit change-point model
3. Detect regression episodes
4. Calculate MTTD

In [ ]:
import sys
sys.path.append('..')

from models.changepoint_model import ChangepointModel
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Generate synthetic time series with change-point
np.random.seed(42)
n_days = 90
changepoint_day = 30

lambda_pre = 0.5  # events per trip
lambda_post = 2.0  # 4x increase

dates = pd.date_range('2024-01-01', periods=n_days)
data = {
    'time': np.arange(n_days),
    'events': np.concatenate([
        np.random.poisson(lambda_pre * 100, changepoint_day),
        np.random.poisson(lambda_post * 100, n_days - changepoint_day)
    ]),
    'exposure': np.random.poisson(100, n_days),
    'dates': dates,
    'start_date': dates[0]
}

print(f"Time series length: {len(data['time'])} days")
print(f"True change-point: Day {changepoint_day}")

In [ ]:
# Visualize time series
plt.figure(figsize=(12, 6))
plt.plot(data['time'], data['events'], 'o-', alpha=0.6)
plt.axvline(changepoint_day, color='red', linestyle='--', label='True Change-point')
plt.xlabel('Day')
plt.ylabel('Event Count')
plt.title('Time Series with Change-point')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Fit model
model = ChangepointModel(samples=1000, tune=500, chains=2)
print("Fitting change-point model...")
idata = model.fit(data, progressbar=True)
print("Model fitting complete!")

In [ ]:
# Detect change-point
detection = model.detect_changepoint(data)
print("Change-Point Detection Results:")
print(f"  Detected: {detection['changepoint_detected']}")
print(f"  Change-point day: {detection['changepoint_time_numeric']:.1f}")
print(f"  Probability: {detection['changepoint_probability']:.2%}")
print(f"  Pre-change hazard: {detection['pre_change_hazard_rate']:.3f}")
print(f"  Post-change hazard: {detection['post_change_hazard_rate']:.3f}")
print(f"  Hazard ratio: {detection['hazard_ratio']:.2f}")
print(f"  MTTD: {detection['mttd_hours']:.1f} hours")